In [ ]:
import transcriber
import json
audio_filepath = "/Users/georgebennett/Documents/GitHub/script_writer/uploads/testsquare.mp3"
audio_file_binary = open(audio_filepath, "rb")
word_segments = transcriber.get_word_segments(audio_file_binary)
print(word_segments)

In [ ]:
# get the list of actual words with their timestamps: 

word_list = transcriber.process_word_segments(word_segments)
print(word_list)

In [ ]:
def fix_missing_time_stamps(word_list):
    """
    sometimes you get this situation:
    {'word': 'money.', 'start': 11.88, 'end': 13.082, 'score': 0.752}
    {'word': '72%'}
    {'word': 'of', 'start': 13.622, 'end': 13.682, 'score': 0.753}

    """

    try:
        fixed_list = []
        for i, word_dict in enumerate(word_list):
            new_dict = word_dict.copy()
            
            # If 'start' is missing in the current word dict
            if 'start' not in new_dict:
                # If it's the first element, set start to 0
                if i == 0:
                    new_dict['start'] = 0
                # Otherwise, set start to end of previous word
                else:
                    new_dict['start'] = fixed_list[-1].get('end', 0)
            
            # If 'end' is missing in the current word dict
            if 'end' not in new_dict:
                # If it's the last element, set end to start + 0.5 (or any other default duration)
                if i == len(word_list) - 1:
                    new_dict['end'] = new_dict['start'] + 0.5
                # Otherwise, set end to start of next word
                else:
                    new_dict['end'] = word_list[i + 1].get('start', new_dict['start'] + 0.5)

            fixed_list.append(new_dict)

        return fixed_list
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [ ]:
def generate_subtitle_blocks(words, char_limit):
    """
    makes blocks like this 
    {'text': 'How to turn your phone into a gold', 'start': None, 'end': 1.61}
    {'text': 'mine,', 'start': 1.63, 'end': 1.85}
    {'text': 'the best kept secret in the world until', 'start': None, 'end': 4.032}
    {'text': 'now.', 'start': 4.112, 'end': 4.333}
    {'text': 'Nope,', 'start': None, 'end': 5.694}
    
    """
    

    # Initialize variables
    blocks = []
    current_block = {"text": "", "start": None, "end": None}

    for word_info in words:
        word = word_info['word']
        start_time = word_info['start']
        end_time = word_info['end']

        # Check if the word ends with punctuation
        ends_in_punctuation = word[-1] in [".", ",", "!", "?", ";"]

        # New block length if this word is added
        new_block_length = len(current_block["text"]) + len(word) + (1 if current_block["text"] else 0)

        # Initialize start time if starting a new block
        if current_block["start"] is None:
            current_block["start"] = start_time

        # Append this word to the current block first, before checking for block completion
        if current_block["text"]:
            current_block["text"] += " "
        current_block["text"] += word
        current_block["end"] = end_time

        if new_block_length > char_limit or ends_in_punctuation:
            # Finalize and save the current block
            blocks.append(current_block.copy())

            # Start a new block
            current_block = {"text": "", "start": None, "end": None}

    # Add the last block if it's not empty
    if current_block["text"]:
        blocks.append(current_block)

    # Display the blocks
    for block in blocks:
        print(block)

    return blocks



subtitle_blocks= generate_subtitle_blocks(fixed_missing_time_stamps, char_limit=10)
# print(subtitle_blocks)

In [ ]:
def process_blocks(block_list):
    # Initialize an empty list to hold the processed blocks
    processed_blocks = []
    
    # Add an empty block that starts at 0 and ends at the start of the original first block
    if block_list:
        processed_blocks.append({'text': '', 'start': 0, 'end': block_list[0]['start']})
    
    # Loop through the list to process each block
    for i, block in enumerate(block_list):
        # Remove the block if it's the first one and the text is empty
        if i == 0 and not block['text'].strip():
            continue
        
        # Make the end time of a block the start time of the next block
        if i < len(block_list) - 1:
            block['end'] = block_list[i + 1]['start']
        
        processed_blocks.append(block)
    
    return processed_blocks


processed_blocks = process_blocks(subtitle_blocks)
print(processed_blocks)

In [ ]:
def seconds_to_srt_time(seconds):
    """Convert seconds to SRT time format (HH:MM:SS,MMM)"""
    hours = int(seconds // 3600)
    seconds %= 3600
    minutes = int(seconds // 60)
    seconds %= 60
    milliseconds = int((seconds % 1) * 1000)
    seconds = int(seconds)
    return f"{hours:02d}:{minutes:02d}:{seconds:02d},{milliseconds:03d}"

def blocks_to_srt(processed_blocks):
    """Convert list of processed blocks to SRT format"""
    srt_content = ""
    for i, block in enumerate(processed_blocks, 1):
        start_time = seconds_to_srt_time(block['start'])
        end_time = seconds_to_srt_time(block['end'])
        text = block['text']
        
        srt_content += f"{i}\n"
        srt_content += f"{start_time} --> {end_time}\n"
        srt_content += f"{text}\n\n"
    
    return srt_content


srt_content = blocks_to_srt(processed_blocks)
print(srt_content)

# To write the content to an SRT file
with open("output.srt", "w") as f:
    f.write(srt_content)


In [20]:
import json
# filepath = "word_segments.json"
with open("word_segments.json", "r") as json_file:
    word_segments = json.load(json_file)



# for item in word_segments:
#     print(item)



import transcriber
word_list = transcriber.process_word_segments(word_segments)
for word in word_list:
    print(word)

{'word': 'Once', 'start': 0.268, 'end': 0.408, 'score': 0.804}
{'word': 'upon', 'start': 0.468, 'end': 0.688, 'score': 0.766}
{'word': 'a', 'start': 0.729, 'end': 0.749, 'score': 1.0}
{'word': 'time,', 'start': 0.809, 'end': 1.029, 'score': 0.76}
{'word': 'Emily', 'start': 1.409, 'end': 1.649, 'score': 0.813}
{'word': 'faced', 'start': 1.709, 'end': 1.989, 'score': 0.785}
{'word': 'the', 'start': 2.009, 'end': 2.109, 'score': 0.834}
{'word': 'colossal', 'start': 2.15, 'end': 2.89, 'score': 0.754}
{'word': 'mountain', 'start': 2.95, 'end': 3.29, 'score': 0.794}
{'word': 'of', 'start': 3.35, 'end': 3.41, 'score': 0.752}
{'word': 'student', 'start': 3.531, 'end': 3.951, 'score': 0.862}
{'word': 'debt,', 'start': 3.991, 'end': 4.251, 'score': 0.952}
{'word': 'feeling', 'start': 4.731, 'end': 5.052, 'score': 0.783}
{'word': 'overwhelmed', 'start': 5.272, 'end': 5.912, 'score': 0.759}
{'word': 'and', 'start': 5.952, 'end': 6.032, 'score': 0.835}
{'word': 'suffocated', 'start': 6.132, 'end': 